In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
timestr = '20201021'

### Building matrix for logistic regression

In [3]:
#fname = 'Records/EXP_03_NMNIST/'+timestr+'_hots_vectorzLR.pkl'
fname = '/tmp/'+timestr+'_hots_vectorzLR_2'

if not os.path.isfile(fname + '_X.npy'):
    
    import pickle
    from HOTS.ToolsMonitor import GenerateActivationMap, DisplayActivationMap
    from HOTS.Event import Event, SimpleAlphabet, LoadNMNIST
    from HOTS.STS import STS
    from HOTS.Layer import ClusteringLayer
    from HOTS.ToolsMonitor import (
        DisplayImage,
        DisplaySurface3D,
        DisplaySurface2D,
        DisplayConvergence,
    )
    from HOTS.Tools import SaveObject, LoadObject
    from HOTS.Network import Network
    from HOTS.Event import conv2eve

    tau = 9e-4 # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
    R = 2
    filthr = 2
    nbkNN = 3
    algo = 'lagorce'
    decay = 'exponential'
    hom = True
    krnlinit = 'rdn'
    nb_cluster = [4, 8, 16]
    ImageSize = (34, 34)
    DataPath = 'Data/testsetnmnist.p'

    NbClusteringData = 15
    NbTrainingData = 40
    NbTestingData = 40
    
    NbClusteringData = 150
    NbTrainingData = 150
    NbTestingData = 40

    #timestr = time.strftime("%Y%m%d")
    hom = False

    if hom==True:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'_homeo.pkl'
    else:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'.pkl'
    print(fname_model)

    _1ofich = False
    event_tr, event_te, event_cl, label_tr, label_te = LoadNMNIST(
    NbTrainingData, NbTestingData, NbClusteringData, OneOfEach=_1ofich, Path=DataPath, OutOnePolarity=False, ListPolarities=None, verbose=0)

    L1 = ClusteringLayer(tau=tau,R=R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    L2 = ClusteringLayer(tau=10 * tau,R=2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    L3 = ClusteringLayer(tau=10 * 10 * tau,R=2 * 2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
    Net = Network([L1, L2, L3])    
    
    if not os.path.isfile(fname_model):
        ClusterLayer, event_output = Net.TrainCluster(
                event=event_cl, NbClusterList=nb_cluster, to_record=True, NbCycle=1
            )
        SaveObject(ClusterLayer, fname_model)
    else: 
        ClusterLayer, Classif0 = LoadObject(fname_model)

    fname_event0_o = '/tmp/'+timestr+'_hots_event0_o.p'
    if not os.path.isfile(fname_event0_o):            
        print('run the events through the network')
        homrun = False
        event0_o_tr2 = Net.RunNetwork(event_tr, NbClusterList=ClusterLayer, homrun=homrun)
        #event0_o_te2 = Net.RunNetwork(event_te, NbClusterList=ClusterLayer, homrun=homrun)
        SaveObject(event0_o_tr2, fname_event0_o)
    else: 
        event0_o_tr2 = LoadObject(fname_event0_o)
    
    # gather data
    n_events = event0_o_tr2.time.shape[0]
    tau_cla = .300 # characteristic time of a digit 
    sample_events = 200
    sample_space = 1
    
    data = np.zeros((ImageSize[0]//sample_space, ImageSize[1]//sample_space, len(event0_o_tr2.ListPolarities))) #tmp data
    data_len = len(data.ravel()) # ImageSize[0]//sample_space*ImageSize[1]//sample_space*len(event0_o_tr2.ListPolarities)
    X = np.zeros((n_events//sample_events, data_len))
    y = np.zeros((n_events//sample_events,))
    
    t_absolute = 0
    t_old = 0
    i_label_tr = 0
    for i in range(1, n_events-1):
        if i == event0_o_tr2.ChangeIdx[i_label_tr]:
            i_label_tr += 1
            t_absolute += event0_o_tr2.time[i]
            
        t_new = t_absolute + event0_o_tr2.time[i]
        data *= np.exp(-(t_new-t_old)/tau_cla)
        t_old = t_new
        
        data[event0_o_tr2.address[i, 0]//sample_space, 
             event0_o_tr2.address[i,1]//sample_space, 
             event0_o_tr2.polarity[i]] = 1.

        if i % sample_events == 0:
            print(f'{i=}')
            X[i//sample_events, :] = data.ravel()
            y[i//sample_events] = label_tr[i_label_tr]

    print('Number of events: '+str(X.shape[0])+' - Number of features: '+str(X.shape[1]))
    #SaveObject([X,y], fname)
    np.save(fname + '_X.npy', X)
    np.save(fname + '_y.npy', y)
    
else: 
    #X,y = LoadObject(fname)
    X = np.load(fname + '_X.npy')
    y = np.load(fname + '_y.npy')
    

Records/EXP_03_NMNIST/20201021_hots_0.9ms_lagorce.pkl
[]
run the events through the network
i=200
i=400
i=600
i=800
i=1000
i=1200
i=1400
i=1600
i=1800
i=2000
i=2200
i=2400
i=2600
i=2800
i=3000
i=3200
i=3400
i=3600
i=3800
i=4000
i=4200
i=4400
i=4600
i=4800
i=5000
i=5200
i=5400
i=5600
i=5800
i=6000
i=6200
i=6400
i=6600
i=6800
i=7000
i=7200
i=7400
i=7600
i=7800
i=8000
i=8200
i=8400
i=8600
i=8800
i=9000
i=9200
i=9400
i=9600
i=9800
i=10000
i=10200
i=10400
i=10600
i=10800
i=11000
i=11200
i=11400
i=11600
i=11800
i=12000
i=12200
i=12400
i=12600
i=12800
i=13000
i=13200
i=13400
i=13600
i=13800
i=14000
i=14200
i=14400
i=14600
i=14800
i=15000
i=15200
i=15400
i=15600
i=15800
i=16000
i=16200
i=16400
i=16600
i=16800
i=17000
i=17200
i=17400
i=17600
i=17800
i=18000
i=18200
i=18400
i=18600
i=18800
i=19000
i=19200
i=19400
i=19600
i=19800
i=20000
i=20200
i=20400
i=20600
i=20800
i=21000
i=21200
i=21400
i=21600
i=21800
i=22000
i=22200
i=22400
i=22600
i=22800
i=23000
i=23200
i=23400
i=23600
i=23800
i=24000
i

i=192400
i=192600
i=192800
i=193000
i=193200
i=193400
i=193600
i=193800
i=194000
i=194200
i=194400
i=194600
i=194800
i=195000
i=195200
i=195400
i=195600
i=195800
i=196000
i=196200
i=196400
i=196600
i=196800
i=197000
i=197200
i=197400
i=197600
i=197800
i=198000
i=198200
i=198400
i=198600
i=198800
i=199000
i=199200
i=199400
i=199600
i=199800
i=200000
i=200200
i=200400
i=200600
i=200800
i=201000
i=201200
i=201400
i=201600
i=201800
i=202000
i=202200
i=202400
i=202600
i=202800
i=203000
i=203200
i=203400
i=203600
i=203800
i=204000
i=204200
i=204400
i=204600
i=204800
i=205000
i=205200
i=205400
i=205600
i=205800
i=206000
i=206200
i=206400
i=206600
i=206800
i=207000
i=207200
i=207400
i=207600
i=207800
i=208000
i=208200
i=208400
i=208600
i=208800
i=209000
i=209200
i=209400
i=209600
i=209800
i=210000
i=210200
i=210400
i=210600
i=210800
i=211000
i=211200
i=211400
i=211600
i=211800
i=212000
i=212200
i=212400
i=212600
i=212800
i=213000
i=213200
i=213400
i=213600
i=213800
i=214000
i=214200
i=214400
i

i=374400
i=374600
i=374800
i=375000
i=375200
i=375400
i=375600
i=375800
i=376000
i=376200
i=376400
i=376600
i=376800
i=377000
i=377200
i=377400
i=377600
i=377800
i=378000
i=378200
i=378400
i=378600
i=378800
i=379000
i=379200
i=379400
i=379600
i=379800
i=380000
i=380200
i=380400
i=380600
i=380800
i=381000
i=381200
i=381400
i=381600
i=381800
i=382000
i=382200
i=382400
i=382600
i=382800
i=383000
i=383200
i=383400
i=383600
i=383800
i=384000
i=384200
i=384400
i=384600
i=384800
i=385000
i=385200
i=385400
i=385600
i=385800
i=386000
i=386200
i=386400
i=386600
i=386800
i=387000
i=387200
i=387400
i=387600
i=387800
i=388000
i=388200
i=388400
i=388600
i=388800
i=389000
i=389200
i=389400
i=389600
i=389800
i=390000
i=390200
i=390400
i=390600
i=390800
i=391000
i=391200
i=391400
i=391600
i=391800
i=392000
i=392200
i=392400
i=392600
i=392800
i=393000
i=393200
i=393400
i=393600
i=393800
i=394000
i=394200
i=394400
i=394600
i=394800
i=395000
i=395200
i=395400
i=395600
i=395800
i=396000
i=396200
i=396400
i

i=556400
i=556600
i=556800
i=557000
i=557200
i=557400
i=557600
i=557800
i=558000
i=558200
i=558400
i=558600
i=558800
i=559000
i=559200
i=559400
i=559600
i=559800
i=560000
i=560200
i=560400
i=560600
i=560800
i=561000
i=561200
i=561400
i=561600
i=561800
i=562000
i=562200
i=562400
i=562600
i=562800
i=563000
i=563200
i=563400
i=563600
i=563800
i=564000
i=564200
i=564400
i=564600
i=564800
i=565000
i=565200
i=565400
i=565600
i=565800
i=566000
i=566200
i=566400
i=566600
i=566800
i=567000
i=567200
i=567400
i=567600
i=567800
i=568000
i=568200
i=568400
i=568600
i=568800
i=569000
i=569200
i=569400
i=569600
i=569800
i=570000
i=570200
i=570400
i=570600
i=570800
i=571000
i=571200
i=571400
i=571600
i=571800
i=572000
i=572200
i=572400
i=572600
i=572800
i=573000
i=573200
i=573400
i=573600
i=573800
i=574000
i=574200
i=574400
i=574600
i=574800
i=575000
i=575200
i=575400
i=575600
i=575800
i=576000
i=576200
i=576400
i=576600
i=576800
i=577000
i=577200
i=577400
i=577600
i=577800
i=578000
i=578200
i=578400
i

IndexError: index 3036 is out of bounds for axis 0 with size 3036

In [ ]:
np.save(fname + '_X.npy', X)
np.save(fname + '_y.npy', y)

In [ ]:
X.shape

In [ ]:
X[:, 3000:5000].max()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.imshow(X[:, 3000:3500])

### Performing logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
lr = LR(random_state=0).fit(X, y)

In [ ]:
lr.score(X, y)

In [ ]:
P_pred = lr.predict_proba(X)
P_pred.shape

In [ ]:
plt.plot(np.max(P_pred, axis=1))

### validation

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression as LR
lr = LR(random_state=0).fit(X_train, y_train)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
help(lr.score)

In [ ]:
P_pred = lr.predict_proba(X_test)
P_pred.shape

In [ ]:
plt.plot(np.max(P_pred, axis=1))